In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from gensim.parsing.preprocessing import PorterStemmer
from gensim.models import Word2Vec
import nltk
nltk.download('reuters') # Downloading corpus
nltk.download('stopwords') # Downloading stopwords
nltk.download('punkt') # Downloading tokenizer

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\nande\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nande\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nande\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

## 2. Use the data.csv dataset as provided as pandas DataFrame and  process the text feature as questioned below.

In [2]:
df = pd.read_csv('data_set.csv')

In [3]:
df.shape

(179, 4)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  179 non-null    int64 
 1   Id          179 non-null    int64 
 2   Text        179 non-null    object
 3   Score       179 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 5.7+ KB


In [5]:
df.describe()

,Unnamed: 0,Id,Score
count,179.000000,179.000000,179.000000
mean,89.000000,288133.765363,3.027933
std,51.816986,163432.883371,1.474236
min,0.000000,3306.000000,1.000000
25%,44.500000,139390.000000,2.000000
50%,89.000000,275801.000000,3.000000
75%,133.500000,423802.500000,4.000000
max,178.000000,568004.000000,5.000000


In [6]:
df

,Unnamed: 0,Id,Text,Score
0,0,3306,"I HATE SUCRALOSE.<br /><br />Honestly, I don't...",3
1,1,7851,I love rice and this was good with all kinds o...,5
2,2,17227,I would like to tell this company that the who...,1
3,3,18152,The label says that this product is made by re...,2
4,4,21017,I ordered the GREENIES 'PETITE' size for my do...,3
...,...,...,...,...
174,174,548557,The bread was a little sweet and it crumbled. ...,3
175,175,557657,Chocolate tastes great! But that goes without ...,4
176,176,559007,"We got a bag of brown, squished Jelly Bellies....",1
177,177,567957,I wanted this for the omega 3's but my daughte...,2


### 2 a) 1.	Remove the accented characters from text feature. (3 marks)

In [7]:
str01 = "Er sagt, dass er beschäftigt ist. = He says he's busy"
str02 = "Ich weiß nicht, ob ich das kann. = I don't know if I can do that."
str03 = "Ich bleibe daheim, weil es regnet. = Ich stay at home because it's raining."
str04 = "Wir essen, bis nichts mehr da ist. = We eat until everything is gone."
unicodedata.normalize('NFKD', str01).encode('ascii', errors='ignore').decode('utf-8', errors='ignore')

"Er sagt, dass er beschaftigt ist. = He says he's busy"

In [61]:
def remove_accented(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', errors='ignore').decode('utf-8', errors='ignore')

print(remove_accented(str01))
print(remove_accented(str02))
print(remove_accented(str03))
print(remove_accented(str04))

Er sagt, dass er beschaftigt ist. = He says he's busy
Ich wei nicht, ob ich das kann. = I don't know if I can do that.
Ich bleibe daheim, weil es regnet. = Ich stay at home because it's raining.
Wir essen, bis nichts mehr da ist. = We eat until everything is gone.


### 2) a) 2)	Remove digits from text feature. (3)

In [62]:
def remove_digits(text):
    re.sub(r"\d+|.\d+", '', text)
    return re.sub(r"\d+", '', text)

string_arr = [ "Er sagt, dass er beschäftigt ist. = He says he's busy 23.47", 
               "Ich weiß nicht, ob ich das kann. = I don't know if I can do 11:29 am 11+29 that.",
               "Ich bleibe daheim, weil es regnet. = Ich stay at home because it's raining for the past 7.59 hours.",
               "Wir essen, bis nichts mehr da ist. = We eat until everything is gone. What will you at 4 pm today for 7 hours" ]

for tmp_str in string_arr:
    print(remove_digits(tmp_str))

Er sagt, dass er beschäftigt ist. = He says he's busy .
Ich weiß nicht, ob ich das kann. = I don't know if I can do : am + that.
Ich bleibe daheim, weil es regnet. = Ich stay at home because it's raining for the past . hours.
Wir essen, bis nichts mehr da ist. = We eat until everything is gone. What will you at  pm today for  hours


# 2) a) 3)	Remove punctuations from text feature. (3)

In [63]:
def remove_punctuation(text):
    text = re.sub(r'[^\w\s]', '', text)
    return text

print(remove_punctuation("I'm working $."))
remove_punctuation("We're working 20$.")

Im working 


'Were working 20'

In [64]:
def remove_stopwords(text):
    swords = stopwords.words('english')
    tokens = str.split(text)
    tokens = [x for x in tokens if x.lower() not in swords]
    return " ".join(tokens)

print(remove_stopwords(str01))
print(remove_stopwords(str02))
print(remove_stopwords(str03))
print(remove_stopwords(str04))

Er sagt, dass er beschäftigt ist. = says he's busy
Ich weiß nicht, ob ich das kann. = know that.
Ich bleibe daheim, weil es regnet. = Ich stay home raining.
Wir essen, bis nichts mehr da ist. = eat everything gone.


In [65]:
def remove_spaces(text):
    text =  re.sub('^ ', '', text)
    text =  re.sub(' $', '', text)
    return re.sub('\s+', ' ', text)

remove_spaces(" You  are     wasting it nicely.     Stop          it. ")

'You are wasting it nicely. Stop it.'

In [66]:
def pre_process(text):
    text = remove_accented(text)
    text = remove_punctuation(text)    
    text = remove_digits(text)
    text = remove_spaces(text)
    text = remove_stopwords(text)
    return text

In [67]:
df['CleanedText'] = df['Text'].apply(pre_process)
df['CleanedText']

0      HATE SUCRALOSEbr br Honestly dont know people ...
1      love rice good kinds textures allergic many th...
2      would like tell company whole box bars melted ...
3      label says product made removing water FAT pea...
4      ordered GREENIES PETITE size dogs done many ti...
                             ...                        
174    bread little sweet crumbled followed direction...
175    Chocolate tastes great goes without sayingI me...
176    got bag brown squished Jelly Bellies Maybe bag...
177    wanted omega daughter wouldnt eat really smell...
178    looks smells great son didnt really like textu...
Name: CleanedText, Length: 179, dtype: object

In [70]:
df['Text'].str.split(expand=True)

,0,1,2,3,4,5,6,7,8,9,...,585,586,587,588,589,590,591,592,593,594
0,I,HATE,SUCRALOSE.<br,/><br,"/>Honestly,",I,don't,know,how,other,...,a,K-Cup,if,you,know,how,to,operate,a,spoon.
1,I,love,rice,and,this,was,good,with,all,kinds,...,None,None,None,None,None,None,None,None,None,None
2,I,would,like,to,tell,this,company,that,the,whole,...,None,None,None,None,None,None,None,None,None,None
3,The,label,says,that,this,product,is,made,by,removing,...,None,None,None,None,None,None,None,None,None,None
4,I,ordered,the,GREENIES,'PETITE',size,for,my,"dogs,",as,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,The,bread,was,a,little,sweet,and,it,crumbled.,I,...,None,None,None,None,None,None,None,None,None,None
175,Chocolate,tastes,great!,But,that,goes,without,saying--I,"mean,",its,...,None,None,None,None,None,None,None,None,None,None
176,We,got,a,bag,of,"brown,",squished,Jelly,Bellies.,Maybe.,...,None,None,None,None,None,None,None,None,None,None
177,I,wanted,this,for,the,omega,3's,but,my,daughter,...,None,None,None,None,None,None,None,None,None,None


In [77]:
list_words = list()
for val in df['CleanedText'].values:
    tokens = val.split()
    for token in tokens:
        list_words.append(token)

df_word = pd.DataFrame(data={"words": list_words})
df_word.groupby(by='words')['words'].agg(['count']).sort_values(by='count', ascending=False).head(10)

,count
words,
like,78
br,65
good,61
taste,52
one,50
flavor,49
product,48
coffee,46
would,39


## c) Vectorize Using Skip-Gram Word2Vec Model

Next, we’ll build and train a skip-gram Word2Vec model and find the 5 most similar words to ‘food’.


In [80]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/satishkumar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [81]:
from gensim.models import Word2Vec

In [83]:
from gensim.models import Word2Vec

# Tokenize the cleaned text
sentences = [text.split() for text in df['CleanedText']]

# Train the skip-gram model
skipgram_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=1)

# Find the 5 most similar words to 'food'
similar_words_skipgram = skipgram_model.wv.most_similar('food', topn=5)
print(similar_words_skipgram)



[('br', 0.9651890397071838), ('like', 0.9612085223197937), ('good', 0.9538864493370056), ('box', 0.9532886743545532), ('Amazon', 0.9527929425239563)]


In [138]:
sentences

[['HATE',
  'SUCRALOSEbr',
  'br',
  'Honestly',
  'dont',
  'know',
  'people',
  'CANT',
  'taste',
  'sweetened',
  'products',
  'got',
  'unpleasant',
  'fake',
  'flavor',
  'cant',
  'ignore',
  'think',
  'company',
  'uses',
  'junk',
  'required',
  'law',
  'indicate',
  'box',
  'advertisingbr',
  'br',
  'said',
  'seems',
  'like',
  'hot',
  'cocoa',
  'less',
  'sucralose',
  'Cafe',
  'Escapes',
  'Hot',
  'Cocoa',
  'give',
  'higher',
  'marks',
  'competitionbr',
  'br',
  'Still',
  'cant',
  'believe',
  'product',
  'nearest',
  'competitor',
  'use',
  'sucralose',
  'dont',
  'say',
  'theyre',
  'diet',
  'lite',
  'lowsugar',
  'otherwise',
  'unnaturallysweetened',
  'product',
  'frustrating',
  'taste',
  'stuff',
  'really',
  'dislike',
  'also',
  'dont',
  'like',
  'chemical',
  'makeup',
  'sucralose',
  'wont',
  'go',
  'believe',
  'natural',
  'sugar',
  'way',
  'better',
  'long',
  'runbr',
  'br',
  'Theres',
  'one',
  'choice',
  'hot',
  '

In [87]:
from gensim.models import Word2Vec

skipgram_model = Word2Vec(sentences=sentences, vector_size=200, window=5, min_count=1, sg=1)
skipgram_model.wv.most_similar('taste', topn=5)

[('br', 0.9761319160461426),
 ('like', 0.9706864953041077),
 ('really', 0.9703561663627625),
 ('flavor', 0.9695756435394287),
 ('would', 0.9691972136497498)]

In [88]:

cbow_model = Word2Vec(sentences=sentences, vector_size=200, window=5, min_count=1, sg=0)
cbow_model.wv.most_similar('taste', topn=5)

[('really', 0.30174946784973145),
 ('rice', 0.27516064047813416),
 ('would', 0.27449220418930054),
 ('get', 0.2660956382751465),
 ('br', 0.24989455938339233)]

In [89]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [91]:
cv = CountVectorizer()
cnt_vectors = cv.fit_transform(df['CleanedText'])
cnt_vectors.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [92]:
tfidf = TfidfVectorizer()
tfidf_vector = tfidf.fit_transform(df['CleanedText'])
tfidf_vector.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [93]:
tfidf_vector.toarray().shape

(179, 2434)

# Build LSTM Model

In [104]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D

In [95]:
df

,Unnamed: 0,Id,Text,Score,CleanedText
0,0,3306,"I HATE SUCRALOSE.<br /><br />Honestly, I don't...",3,HATE SUCRALOSEbr br Honestly dont know people ...
1,1,7851,I love rice and this was good with all kinds o...,5,love rice good kinds textures allergic many th...
2,2,17227,I would like to tell this company that the who...,1,would like tell company whole box bars melted ...
3,3,18152,The label says that this product is made by re...,2,label says product made removing water FAT pea...
4,4,21017,I ordered the GREENIES 'PETITE' size for my do...,3,ordered GREENIES PETITE size dogs done many ti...
...,...,...,...,...,...
174,174,548557,The bread was a little sweet and it crumbled. ...,3,bread little sweet crumbled followed direction...
175,175,557657,Chocolate tastes great! But that goes without ...,4,Chocolate tastes great goes without sayingI me...
176,176,559007,"We got a bag of brown, squished Jelly Bellies....",1,got bag brown squished Jelly Bellies Maybe bag...
177,177,567957,I wanted this for the omega 3's but my daughte...,2,wanted omega daughter wouldnt eat really smell...


In [106]:
X = df['CleanedText']
y = pd.get_dummies(df['Score'], dtype='int32')
y
max_words = 5000
max_len = 100

tokenizer = Tokenizer(num_words=max_words, lower=True)
tokenizer.fit_on_texts(X)
X_tokenized = tokenizer.texts_to_sequences(X)
X_token_padded = pad_sequences(X_tokenized, maxlen=max_len)

X_train, X_test, y_train, y_test = train_test_split(X_token_padded, y, test_size=0.3, random_state=43)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((125, 100), (54, 100), (125, 5), (54, 5))

In [109]:
y

,1,2,3,4,5
0,0,0,1,0,0
1,0,0,0,0,1
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
...,...,...,...,...,...
174,0,0,1,0,0
175,0,0,0,1,0
176,1,0,0,0,0
177,0,1,0,0,0


In [107]:
X_train

array([[   0,    0,    0, ...,  908,  728,    5],
       [   0,    0,    0, ..., 1471,   73,  178],
       [   0,    0,    0, ...,  285,  484,  131],
       ...,
       [   0,    0,    0, ...,   44,  224,    5],
       [   0,    0,    0, ...,    2,  539,    1],
       [   0,    0,    0, ...,  843,  844,   45]], dtype=int32)

In [112]:
# Build the LSTM model
model = Sequential()
model.add(Embedding(max_words, 128))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5, activation='softmax'))  # 3 classes for sentiment

In [113]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [115]:
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), verbose=2)

Epoch 1/10
2/2 - 0s - 69ms/step - accuracy: 0.6240 - loss: 1.5309 - val_accuracy: 0.3333 - val_loss: 1.5995
Epoch 2/10
2/2 - 0s - 55ms/step - accuracy: 0.5360 - loss: 1.4991 - val_accuracy: 0.2963 - val_loss: 1.6120
Epoch 3/10
2/2 - 0s - 55ms/step - accuracy: 0.5040 - loss: 1.4796 - val_accuracy: 0.3148 - val_loss: 1.6007
Epoch 4/10
2/2 - 0s - 57ms/step - accuracy: 0.6080 - loss: 1.4244 - val_accuracy: 0.3704 - val_loss: 1.5931
Epoch 5/10
2/2 - 0s - 55ms/step - accuracy: 0.6480 - loss: 1.3743 - val_accuracy: 0.3519 - val_loss: 1.5960
Epoch 6/10
2/2 - 0s - 55ms/step - accuracy: 0.6480 - loss: 1.3154 - val_accuracy: 0.3519 - val_loss: 1.6111
Epoch 7/10
2/2 - 0s - 55ms/step - accuracy: 0.6480 - loss: 1.2342 - val_accuracy: 0.3519 - val_loss: 1.6241
Epoch 8/10
2/2 - 0s - 56ms/step - accuracy: 0.6240 - loss: 1.1720 - val_accuracy: 0.3333 - val_loss: 1.6166
Epoch 9/10
2/2 - 0s - 56ms/step - accuracy: 0.7280 - loss: 1.0784 - val_accuracy: 0.3333 - val_loss: 1.6003
Epoch 10/10
2/2 - 0s - 55ms/

In [116]:
# Display the model summary
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 100, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_2             │ (None, 100, 128)       │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 100)            │        91,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           505 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,196,317 (8.38 MB)

 Trainable params: 732,105 (2.79 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,464,212 (5.59 MB)

In [122]:
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

# Predict on the validation set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


In [123]:
y_pred_classes, y_true

(array([4, 0, 4, 0, 3, 4, 3, 0, 4, 0, 2, 4, 2, 0, 0, 0, 4, 3, 4, 4, 0, 3,
        4, 3, 0, 3, 4, 0, 4, 4, 3, 0, 3, 4, 4, 0, 2, 4, 0, 0, 4, 4, 3, 4,
        0, 0, 4, 0, 0, 0, 2, 1, 3, 0]),
 array([1, 0, 0, 4, 2, 4, 2, 0, 4, 2, 3, 0, 0, 1, 2, 2, 1, 2, 1, 3, 0, 4,
        4, 3, 4, 4, 1, 0, 0, 4, 0, 0, 3, 1, 4, 2, 4, 2, 4, 0, 4, 4, 2, 3,
        0, 3, 0, 0, 1, 1, 3, 2, 4, 1]))

In [124]:
confusion_matrix(y_true, y_pred_classes)

array([[8, 0, 1, 1, 4],
       [4, 0, 0, 0, 5],
       [4, 1, 0, 4, 1],
       [1, 0, 2, 2, 2],
       [3, 0, 1, 3, 7]])

In [125]:
from sklearn.metrics import classification_report

In [126]:
rpt = classification_report(y_true, y_pred_classes)
print(rpt)

              precision    recall  f1-score   support

           0       0.40      0.57      0.47        14
           1       0.00      0.00      0.00         9
           2       0.00      0.00      0.00        10
           3       0.20      0.29      0.24         7
           4       0.37      0.50      0.42        14

    accuracy                           0.31        54
   macro avg       0.19      0.27      0.23        54
weighted avg       0.23      0.31      0.26        54



In [128]:
# Predict on the validation set
y_pred = model.predict(X_token_padded)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y, axis=1)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


In [129]:
y_pred_classes

array([0, 4, 0, 4, 2, 1, 1, 3, 1, 4, 0, 0, 2, 2, 3, 3, 1, 4, 3, 4, 2, 0,
       0, 0, 0, 4, 3, 3, 3, 3, 1, 3, 4, 0, 3, 4, 4, 3, 4, 4, 4, 4, 2, 4,
       4, 3, 0, 3, 3, 4, 0, 3, 4, 1, 0, 3, 1, 4, 0, 4, 2, 0, 2, 4, 3, 0,
       0, 4, 0, 0, 2, 4, 3, 0, 2, 4, 4, 0, 0, 4, 2, 0, 4, 4, 1, 0, 2, 3,
       0, 2, 0, 4, 3, 3, 4, 3, 0, 0, 0, 1, 4, 1, 3, 3, 2, 2, 3, 0, 0, 2,
       0, 0, 0, 4, 0, 1, 1, 1, 4, 3, 0, 3, 1, 2, 0, 3, 1, 1, 0, 0, 3, 0,
       3, 4, 2, 0, 4, 0, 4, 3, 4, 4, 3, 0, 0, 0, 0, 4, 3, 2, 4, 4, 2, 3,
       4, 3, 2, 2, 4, 0, 4, 4, 4, 2, 0, 3, 3, 4, 4, 0, 0, 3, 0, 3, 3, 3,
       0, 3, 4])

In [130]:
confusion_matrix(y_true, y_pred_classes)

array([[34,  0,  1,  1,  4],
       [ 4, 15,  0,  4,  7],
       [ 9,  1, 18,  6,  1],
       [ 1,  0,  2, 28,  2],
       [ 3,  0,  1,  3, 34]])

In [133]:
rpt = classification_report(y_true, y_pred_classes)
print(rpt)

              precision    recall  f1-score   support

           0       0.67      0.85      0.75        40
           1       0.94      0.50      0.65        30
           2       0.82      0.51      0.63        35
           3       0.67      0.85      0.75        33
           4       0.71      0.83      0.76        41

    accuracy                           0.72       179
   macro avg       0.76      0.71      0.71       179
weighted avg       0.75      0.72      0.71       179



In [136]:
def remove_html_tags(text):
    return re.sub( r'<.*?>', '', text)

html_text = "<html><head>Header</head><body>sample text</body></html>"

remove_html_tags(html_text)

'Headersample text'